In [404]:
# Below, I try to do some text processing and tokenization to see word frequencies

import codecs
import re
import copy
import collections

import numpy as np
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import WordPunctTokenizer
from __future__ import division
# import matplotlib
import json

import json


with open('Master_indeed_dict.json') as json_file:
    Master_indeed_dict = json.load(json_file)
with open('USAJ_tokenized_dict.json') as json_file:
    USAJ_tokenized_dict = json.load(json_file)
with open('syllabi_NICCS.json') as json_file:
    syllabi_NICCS_dict = json.load(json_file)    
with open('Certs.json') as json_file:
    Certs_dict = json.load(json_file)  
with open('similar.json') as json_file:
    similar = json.load(json_file)

In [417]:
from sklearn.metrics.pairwise import cosine_similarity   

def vectorize(list1):
    vecs_1 = []
    for sent in list1:
        vecs_1.append(model.encode(sent))
    return vecs_1

def Alignment(list1, list2):
    alignment = {}
    tmp = []
    for sentence in list1:
        # if type(sentence) is list:
        #     print("list")
        # if type(sentence) is str:
        sim = cosine_similarity([sentence], list2[:])
        # mean C_s of 1 sentence in list1 vs all sentences in list2
        tmp.append(np.mean(sim))

        alignment['cos_all'] = tmp
        alignment['cos_mean'] = np.mean(tmp) if len(tmp)>=1 else 0
        tmp_ = []
        for val in tmp:
            tmp_.append(1-np.arccos(val)/np.pi) #converts into angle and normalize [-1 1]
        alignment['angle'] = tmp_ 
        alignment['angle_mean'] = np.mean(tmp_) if len(tmp_)>=1 else 0
        
    return alignment
    

def Compare(dict1, dict2, vec=False, baseline=False):
   Sim = {}
   if baseline:
       for K in dict1.keys():
            Sim[K] = {}
            vec_1 = dict1[K]['all_vec'] if vec else vectorize(dict1[K]['All'])
            # for k in dict2.keys():
            #     vec_2 = dict2[k]['all_vec'] if vec else vectorize(dict2[k]['All'])
            try:
                Sim[K][K] = Alignment(vec_1, vec_1)
            except:
                    pass
   else:
       for K in dict1.keys():
        Sim[K] = {}
        vec_1 = dict1[K]['all_vec'] if vec else vectorize(dict1[K]['All'][:2])
        for k in dict2.keys():
            vec_2 = dict2[k]['all_vec'] if vec else vectorize(dict2[k]['All'][:2])
            try:
                    Sim[K][k] = Alignment(vec_1, vec_2)
            except:
                    pass
        
   return Sim
   
   

def Score(baseline, mean):
    return 1-np.abs((baseline-mean)/mean)

def Calculate_Score(dict1, baseline_dict, key=False):
    list1 = []
    for K in dict1.keys():
        baseline = baseline_dict[K][K]['cos_mean']
        for k in dict1[K].keys():
            if not key:
                mean = dict1[K][k]['cos_mean']
                list1.append(Score(baseline,mean))
            else:
                baseline = baseline_dict[K][K][key]
                mean = dict1[K][k][key]
                list1.append(Score(baseline,mean))
    return list1

In [ ]:
from sentence_transformers import SentenceTransformer, util
# model = SentenceTransformer('bert-base-nli-mean-tokens')
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [ ]:
#####################
## HERE I want to vectorize each sentence while preserving the data structure (instead of a sentence, we will have a vector/list)
syllabi_niccs_vecs = {}
for WR in syllabi_NICCS_dict.keys():
    all_vec = model.encode(syllabi_NICCS_dict[WR]['All'])
    k_vec = model.encode(syllabi_NICCS_dict[WR]['K'])
    s_vec = model.encode(syllabi_NICCS_dict[WR]['S'])
    a_vec = model.encode(syllabi_NICCS_dict[WR]['A'])
    
    syllabi_niccs_vecs[WR] = {'all_vec': all_vec,
                            'K': k_vec, 
                             'S': s_vec, 
                             'A': a_vec}

In [ ]:
#####################
# USAJobs
## HERE I want to vectorize each sentence while preserving the data structure (instead of a sentence, we will have a vector/list)
usaJ_vecs = {}
for job in USAJ_tokenized_dict.keys():
    all_vec = model.encode(USAJ_tokenized_dict[job]['All'])
    k_vec = model.encode(USAJ_tokenized_dict[job]['K'])
    s_vec = model.encode(USAJ_tokenized_dict[job]['S'])
    a_vec = model.encode(USAJ_tokenized_dict[job]['A'])
    
    usaJ_vecs[job] = {'all_vec': all_vec, 'K': k_vec, 
                               'S': s_vec, 
                               'A': a_vec}

In [ ]:
#####################
## HERE I want to vectorize each sentence while preserving the data structure (instead of a sentence, we will have a vector/list)
Indeed_vecs = {}
for job in Master_indeed_dict.keys():
    all_vec = model.encode(Master_indeed_dict[job]['All'])
    k_vec = model.encode(Master_indeed_dict[job]['K'])
    s_vec = model.encode(Master_indeed_dict[job]['S'])
    a_vec = model.encode(Master_indeed_dict[job]['A'])
    
    Indeed_vecs[job] = {'all_vec': all_vec, 'K': k_vec, 
                               'S': s_vec, 
                               'A': a_vec}

In [ ]:
#####################
## HERE I want to vectorize each sentence while preserving the data structure (instead of a sentence, we will have a vector/list)
Certs_vecs = {}
for cert in Certs_dict.keys():
    all_vec = model.encode(Certs_dict[cert]['All'])
    k_vec = model.encode(Certs_dict[cert]['K'])
    s_vec = model.encode(Certs_dict[cert]['S'])
    a_vec = model.encode(Certs_dict[cert]['A'])
    
    Certs_vecs[cert] = {'all_vec': all_vec,'K': k_vec, 
                        'S': s_vec, 
                        'A': a_vec}

In [ ]:
# ### New Approach:
# # Well, this approach is not really new since it was the obvious thing to do. My first initial try at comparing
# # similarities between job postings and the other datasets, I first tried to categorize the job titles under one of the 
# # 52 work roles defined by NIST; I even tried to group them manually by going through the job titles and descriptions to see
# # where they fit. Neverthless, due to my limited knowledge about cyber-security jobs, job titles that list two different work 
# # roles separated by a '/' and the tidious manual work, it seemed to be impossible.
# # Therefore, I just decided to compare each workrole to every single job posting.
 

In [ ]:
# Calculating the baselines
baseline_in = Compare(Indeed_vecs, Indeed_vecs, vec=True,baseline=True)
baseline_us = Compare(usaJ_vecs, usaJ_vecs, vec=True,baseline=True)
baseline_sy = Compare(syllabi_niccs_vecs, syllabi_niccs_vecs, vec=True,baseline=True)

In [ ]:
# Syllabi VS USAJobas

a = Compare(usaJ_vecs, syllabi_niccs_vecs, vec=True)
a_1 = Compare(syllabi_niccs_vecs, usaJ_vecs, vec=True)

US_Sy_cos = Calculate_Score(a, baseline_us)
# print(np.mean(US_Sy_cos))
SY_US_cos = Calculate_Score(a_1, baseline_sy)
print(np.mean(SY_US_cos+US_Sy_cos))

In [ ]:
# Indeed Vs Syllabi

b = Compare(Indeed_vecs, syllabi_niccs_vecs, vec=True)
b_1 = Compare(syllabi_niccs_vecs, Indeed_vecs, vec=True)

I_Sy_cos = Calculate_Score(b, baseline_in)
# print(np.mean(I_Sy_cos))
SY_I_cos = Calculate_Score(b_1, baseline_sy)
print(np.mean(SY_I_cos+I_Sy_cos))

In [ ]:
# Indeed VS USAJobs

c = Compare(Indeed_vecs, usaJ_vecs, vec=True)
c_2 = Compare(usaJ_vecs, Indeed_vecs, vec=True)

I_US_cos = Calculate_Score(c, baseline_in)
US_I_cos = Calculate_Score(c_2, baseline_us)

print(np.mean(US_I_cos+I_US_cos))